In [1]:
import tkinter as tk
from tkinter import filedialog

import scipy.io
import os
import numpy as np

import gc

from funcs import vhdr_to_mne, bipolar_montage, align, prepare_data, concatenation,load_sorciere_regressors, model_definition, trf_spyeeg

## paths

In [2]:
# first 'select a folder direction 'patient espace''
# then, select the sorciere file in the NAS (sorcière = file.vhdr)
# then, select the pianos files in the NAS (pianos = [file.vhdr, file.vhdr..]
# Fonction pour sélectionner le chemin vers le dossier "sorciere"
def select_sorciere_path():
    global sorciere_path
    sorciere_path = filedialog.askopenfilenames()
    if sorciere_path:
        print("Fichiers sélectionnés:")
        for file in sorciere_path:
            print(file)


# Fonction pour sélectionner le chemin vers le dossier de sortie
def select_out_path():
    global out_path
    out_path = filedialog.askdirectory()
    if out_path:
        print("Chemin vers le dossier de sortie sélectionné:", out_path)

# Création de la fenêtre principale
root = tk.Tk()
root.title("Ouvrir")

# Déclaration des variables pour stocker les chemins sélectionnés
sorciere_path = ""
piano_path = ""
out_path = ""

# Boutons pour sélectionner les chemins vers les dossiers
sorciere_button = tk.Button(root, text="Select Sorcière *.vhdr (on NAS)", command=select_sorciere_path)
sorciere_button.pack(pady=5)

out_button = tk.Button(root, text="Select patient workspace (locally)", command=select_out_path)
out_button.pack(pady=5)

root.mainloop()

# Vous pouvez maintenant utiliser les variables sorciere_path, piano_path et out_path dans votre application.


Fichiers sélectionnés:
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_12_Tor_Eli/11.04/Sorcière/250411C-A_0007.vhdr
Chemin vers le dossier de sortie sélectionné: C:/Users/nadege/Desktop/camille/seeg/agnes/pres


## Import .vhdr to mne

In [ ]:
sorciere_mne = vhdr_to_mne(sorciere_path)

/

/

1
1
3
3
9
9
.
.
1
1
2
2
4
4
.
.
1
1
4
4
8
8
.
.
1
1
2
2
3
3
/

s
s
e
e
e
e
g
g
/

S
S
e
e
e
e
g
g
_
_
1
1
9
9
9
9
8
8
-
-
2
2
0
0
2
2
5
5
/

S
S
e
e
e
e
g
g
2
2
0
0
2
2
5
5
/

s
s
E
E
E
E
G
G
_
_
1
1
2
2
_
_
T
T
o
o
r
r
_
_
E
E
l
l
i
i
/

1
1
1
1
.
.
0
0
4
4
/

S
S
o
o
r
r
c
c
i
i
è
è
r
r
e
e


AttributeError: 'list' object has no attribute 'get_montage'

In [ ]:
sorciere_path, file = os.path.split(sorciere_path[0])
del file

In [5]:
stim_sorcière = [file for file in os.listdir(sorciere_path+'/stim') if os.path.isfile(os.path.join(sorciere_path+'/stim', file)) and file.endswith('.mp3')]

In [6]:
stim_sorcière

['sorciere_2ms_trig.mp3']

## Montage bipolaire

Do a bipolar montage en soustrayant l'activité de deux shaft consécutives /!\ de le même electrode

In [7]:
sorciere_bipol, channels_bipol = bipolar_montage(sorciere_mne[0], chans=True)
sorciere_bipol = [sorciere_bipol]


In [ ]:
#Trier les channels dans l'ordre alphabétique et numérique
def custom_sort(value):
    # Séparer la partie alphabétique et numérique
    alpha_part = ''.join(filter(lambda x: x.isalpha() or x == "'", value))
    num_part = int(''.join(filter(str.isdigit, value)))
    return alpha_part, num_part

sorted_channels_bipol = sorted(channels_bipol, key=custom_sort)

# Sorciere
sorciere_bipol_sorted = [np.empty_like(sorciere_bipol[0])]
for i, channel_name in enumerate(sorted_channels_bipol):
    channel_index = channels_bipol.index(channel_name)
    sorciere_bipol_sorted[0][:, i] = sorciere_bipol[0][:, channel_index]


In [35]:
scipy.io.savemat(out_path+'/channels.mat', {'ch':np.array(sorted_channels_bipol)})

In [9]:
import mne
from pydub import AudioSegment
from collections import Counter
import matplotlib.pyplot as plt
from scipy.signal import correlate

def align_origin(raw_fif, raw_array, stim_dir) :
  stim_name = os.path.basename(stim_dir)

  #load .mp3
  aud = AudioSegment.from_file(stim_dir, format="mp3")
  fs_stim = aud.frame_rate

  # stereo format : audio/trigger
  chans = aud.split_to_mono()
  trigg = chans[1].get_array_of_samples()  # triggers

  # binarized triggers
  thrsh_exceed_idx = []
  seuil = 3000
  for idx, valeur in enumerate(trigg):
    if valeur > seuil and trigg[idx - 1] < seuil and trigg[idx - 2] < seuil:
      thrsh_exceed_idx.append(idx)

  trigg_stim = np.array(thrsh_exceed_idx)
  trigg_stim = trigg_stim / fs_stim

  # load data
  fs_data = raw_fif.info['sfreq']

  # find events
  orig_events, events_dict = mne.events_from_annotations(raw_fif)
  #take only trigg of interest
  counter = Counter(orig_events[:, 2])
  t_int = counter.most_common(1)[0][0]
  trigg_data_samples = np.squeeze(orig_events[np.where(orig_events[:, 2] == t_int), 0])
  #trigg_data_samples = trigg_data_samples[5:]
  trigg_data_secondes = trigg_data_samples / fs_data

  plt.figure()
  plt.vlines(trigg_data_secondes, 0,1, label = "trigg_data")
  plt.vlines(trigg_stim, 1,2, color='orange', label = "trigg_stim")
  plt.xlim(0,10)
  plt.legend()
  plt.title("before alignement")

  print('data :', len(trigg_data_secondes))
  print('stim :', len(trigg_stim))

  i=0
  max_iteration = 150
  while True:
    if len(trigg_data_secondes) == len(trigg_stim):
      print('same numbre of trigger')
      trigg_stim_zero = trigg_stim - trigg_stim[0]
      trigg_data_zero = trigg_data_secondes - trigg_data_secondes[0]

    elif len(trigg_data_secondes) < len(trigg_stim):
      print('< number of trigger')
      if i >= len(trigg_stim):  # Si i dépasse la taille de trigg_stim
          print(f"Stopping: i ({i}) exceeded trigg_stim size ({len(trigg_stim)})")
          break
      trigg_stim_zero = trigg_stim - trigg_stim[i]
      trigg_data_zero = trigg_data_secondes - trigg_data_secondes[0]

    elif len(trigg_data_secondes) > len(trigg_stim):
      print('> number of trigger')
      if i >= len(trigg_data_secondes):  # Si i dépasse la taille de trigg_data_secondes
          print(f"Stopping: i ({i}) exceeded trigg_data_secondes size ({len(trigg_data_secondes)})")
          break
      trigg_stim_zero = trigg_stim - trigg_stim[0]
      trigg_data_zero = trigg_data_secondes - trigg_data_secondes[i]

    # calcul delay between all triggs
    matrice_differences = np.zeros((len(trigg_data_zero), len(trigg_stim_zero)))
    for j in range(len(trigg_data_zero)):
      for k in range(len(trigg_stim_zero)):
        matrice_differences[j, k] = trigg_data_zero[j] - trigg_stim_zero[k]

    # take only closest pairs of trigger
    if len(trigg_data_zero) <= len(trigg_stim_zero):
      trigg_stim_zero_syncr = []
      for j in range(len(trigg_data_zero)):
        idx_syncr = np.argmin(np.abs(matrice_differences[j, :]))
        trigg_stim_zero_syncr.append(trigg_stim_zero[idx_syncr])
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero), len(trigg_stim_zero_syncr)))
      for j in range(len(trigg_data_zero)):
        for k in range(len(trigg_stim_zero_syncr)):
          matrice_differences_[j, k] = trigg_data_zero[j] - trigg_stim_zero_syncr[k]
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero), len(trigg_stim_zero_syncr)))
      for j in range(len(trigg_data_zero)):
        for k in range(len(trigg_stim_zero_syncr)):
          matrice_differences_[j, k] = trigg_data_zero[j] - trigg_stim_zero_syncr[k]

    elif len(trigg_data_zero) > len(trigg_stim_zero):
      trigg_data_zero_syncr = []
      for j in range(len(trigg_stim_zero)):
        idx_syncr = np.argmin(np.abs(matrice_differences[:, j]))
        trigg_data_zero_syncr.append(trigg_data_zero[idx_syncr])
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero_syncr), len(trigg_stim_zero)))
      for j in range(len(trigg_data_zero_syncr)):
        for k in range(len(trigg_stim_zero)):
          matrice_differences_[j, k] = trigg_data_zero_syncr[j] - trigg_stim_zero[k]
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero_syncr), len(trigg_stim_zero)))
      for j in range(len(trigg_data_zero_syncr)):
        for k in range(len(trigg_stim_zero)):
          matrice_differences_[j, k] = trigg_data_zero_syncr[j] - trigg_stim_zero[k]

    # mean delay
    tmp = np.mean(np.diagonal(matrice_differences_))
    print(np.abs(round(tmp, 4)))
    if np.abs(round(tmp, 4)) > 0.005 :
      i+=1
      continue

    else :
      if i == 0 :
        raw_array_ = raw_array[trigg_data_samples[0]:trigg_data_samples[-1],:]
        plt.figure()
        plt.vlines(trigg_data_zero, 0,1, label = "trigg_data")
        plt.vlines(trigg_stim_zero, 1,2, color='orange', label = "trigg_stim")
        plt.xlim(-1,10)
        plt.legend()
        plt.title("after alignement")
      else :
        if len(trigg_data_secondes) < len(trigg_stim):
          raw_array_ = raw_array[trigg_data_samples[0]:trigg_data_samples[-1],:]
          zrs = int((trigg_stim[i]-trigg_stim[0])*fs_data)
          add_zeros = np.zeros((zrs, raw_array_.shape[1]))
          raw_array_ = np.concatenate((add_zeros, raw_array_), axis=0)

        elif len(trigg_data_secondes) > len(trigg_stim):
          raw_array_ = raw_array[trigg_data_samples[i]:trigg_data_samples[-1],:]
      break

  return raw_array_

## Align

In [26]:
sorted_channels_bipol

["A'1-A'2",
 "A'2-A'3",
 "A'3-A'4",
 "A'4-A'5",
 "A'5-A'6",
 "A'6-A'9",
 "A'9-A'10",
 "A'10-A'11",
 "A'11-A'12",
 "A'12-A'13",
 "B'1-B'2",
 "B'2-B'3",
 "B'3-B'4",
 "B'4-B'5",
 "B'5-B'6",
 "B'6-B'7",
 "B'7-B'8",
 "B'8-B'9",
 "B'9-B'10",
 "B'10-B'11",
 'B1-B2',
 'B2-B3',
 'B3-B4',
 'B4-B5',
 'B5-B6',
 'B6-B7',
 'B7-B8',
 'B8-B9',
 "CR'1-CR'2",
 "CR'2-CR'3",
 "CR'3-CR'4",
 "CR'4-CR'5",
 "CR'5-CR'6",
 "CR'6-CR'7",
 "CR'7-CR'8",
 "CR'8-CR'9",
 "CR'9-CR'10",
 "CR'10-CR'11",
 "CR'11-CR'12",
 "CR'12-CR'13",
 "GC'1-GC'2",
 "GC'2-GC'3",
 "GC'3-GC'4",
 "GC'4-GC'5",
 "GC'5-GC'6",
 "GC'6-GC'7",
 "GC'7-GC'8",
 "GC'8-GC'9",
 "GC'9-GC'10",
 "GC'10-GC'11",
 "GC'11-GC'12",
 "GC'12-GC'13",
 "GPH'1-GPH'2",
 "GPH'2-GPH'3",
 "GPH'3-GPH'4",
 "GPH'4-GPH'5",
 "GPH'5-GPH'6",
 "GPH'6-GPH'7",
 "GPH'7-GPH'8",
 "GPH'8-GPH'9",
 "GPH'9-GPH'10",
 "GPH'10-GPH'11",
 "H'1-H'2",
 "H'2-H'3",
 "H'3-H'4",
 "H'4-H'5",
 "H'5-H'6",
 "H'6-H'7",
 "H'7-H'8",
 "H'8-H'9",
 "H'9-H'10",
 "H'10-H'11",
 "H'11-H'12",
 "H'12-H'13",
 "H'13

In [11]:
%matplotlib qt
sorciere_aligned = align(raw_fif=sorciere_mne[0], raw_array=sorciere_bipol_sorted[0], stim_dir=os.path.join(sorciere_path,'stim', stim_sorcière[0]))


Used Annotations descriptions: ['Comment/?', 'Comment/D?but', 'Stimulus/S 128', 'Stimulus/S 192', 'Stimulus/S 200']
data : 582
stim : 585
< number of trigger
0.0027


## Filtering 0.1-8Hz - Z-scoring - Downsampling

In [12]:
high_pass = 0.1
low_pass = 35

fs_data = sorciere_mne[0].info['sfreq']

pianos_downsampled = []

sorciere_ultimate = prepare_data(data= sorciere_aligned, picks=sorted_channels_bipol, high_pass=high_pass, low_pass=low_pass, fs_data=fs_data)

## Concatenate Pianos & Load Pianos regressors

## Load Sorcière regressors

In [13]:
import sys
sys.path.append(r"C:\Users\nadege\mambaforge\Lib\site-packages")
import textgrids
print("textgrids importé avec succès.")


textgrids importé avec succès.


In [14]:
sorciere_ultimate, sorciere_regs_all, sorciere_regs_all_label = load_sorciere_regressors(sorciere_path, sorciere_ultimate)

## Define models

In [15]:
# Sorcière
## loudness, brighness, periodicity + derivatives_hr + onset syllabe
model_acoustic = [0, 1, 3, 12, 13, 15, 18]

## acoustic + onset_word + surprise + entropy
model_predictif = [0, 1, 3, 12, 13, 15, 18, 19, 20]

sorciere_regs_acoustic, sorciere_regs_acoustic_label, sorciere_regs_predictive, sorciere_regs_predictive_labels = model_definition(sorciere_regs_all, sorciere_regs_all_label, model_acoustic, model_predictif)


In [31]:
sorciere_regs_predictive_labels

['loudness',
 'brightness',
 'periodicity',
 'half_rectified_der_loudness',
 'half_rectified_der_brightness',
 'half_rectified_der_periodicity',
 'syllabe_onset',
 'onsets',
 'entropy']

## TRF

In [16]:
from datetime import datetime

start_at = datetime.now()

In [19]:
fs = 100

In [20]:
n_folds = 5
tw_trf = [-2, 2]  #in sec

coefs_times = np.linspace(tw_trf[0], tw_trf[1], int(np.diff(tw_trf) * fs + 1))  #get TRF time-axis

alphas = [1e-4, 1e-2, 1e2, 1e4, 1e6, 1e8]

### Sorcière : acoustic TRF

In [21]:
import sys
sys.path.append(r"C:\Users\nadege\mambaforge\Lib\site-packages")
import spyeeg
print("Bibliothèque importée avec succès.")

Bibliothèque importée avec succès.


array([[[-0.93146294, -0.71820297, -1.04636823, ..., -0.46331206,
         -0.57505556,  0.        ]],

       [[-1.69609743, -2.39421712, -1.1731964 , ..., -0.46331206,
         -0.57505556,  0.        ]],

       [[-1.50710331, -1.97501603, -1.15277995, ...,  1.36111533,
         -0.4734942 ,  0.        ]],

       ...,

       [[ 0.62345085, -0.73698744,  0.85280011, ...,  0.53137815,
         -0.57505556,  0.        ]],

       [[ 0.82475343, -0.65446351,  0.98321863, ..., -0.10415527,
          0.07370962,  0.        ]],

       [[ 0.88995856, -0.60196518,  0.90492382, ..., -0.23483127,
         -0.57505556,  0.        ]]])

In [22]:
sorciere_regs_acoustic_scores_cv, sorciere_regs_acoustic_coefs_fit = trf_spyeeg(
                                                    regs = np.squeeze(sorciere_regs_acoustic),
                                                    data = np.squeeze(sorciere_ultimate),
                                                    tmin=tw_trf[0], tmax=tw_trf[1],
                                                    fs=fs,
                                                    alpha=alphas,
                                                    channels=sorted_channels_bipol)


# save r2 in .mat format
scipy.io.savemat(out_path+'/sorciere_acoustic.mat',
                 {'data' : sorciere_ultimate,
                  'channels': sorted_channels_bipol,
                  'r2': sorciere_regs_acoustic_scores_cv,
                  'kernels':sorciere_regs_acoustic_coefs_fit,
                  'fs': fs})

gc.collect()

200

### Sorcière : predictive TRF

In [23]:
sorciere_regs_predictive_scores_cv, sorciere_regs_predictive_coefs_fit = trf_spyeeg(
regs = np.squeeze(sorciere_regs_predictive),
data = np.squeeze(sorciere_ultimate),
tmin=tw_trf[0], tmax=tw_trf[1],
fs=fs,alpha=alphas,channels=sorted_channels_bipol)

# save r2 in .mat format
scipy.io.savemat(out_path+'/sorciere_predictive.mat',
                 {'data' : sorciere_ultimate,
                  'channels': sorted_channels_bipol,
                  'r2': sorciere_regs_predictive_scores_cv,
                  'kernels':sorciere_regs_predictive_coefs_fit,
                  'fs': fs})

gc.collect()

0

In [ ]:
import numpy as np
import pickle
import pandas as pd

# ========================
# PARAMS
# ========================


r2 = sorciere_regs_acoustic_scores_cv  # ou predictive selon ce que tu veux exporter
channels = sorted_channels_bipol
locations_subject = locations[subject]  # array (3, n_channels)

# ========================
# THRESHOLD
# ========================
percentiles_gros = [1, 99]
percentiles_seuil_r2 = 90
thresh_type = "mean"
standards = 0

perc = np.percentile(r2, percentiles_gros)
filtered = r2[r2 >= perc[0]]

if thresh_type == "median":
    threshold = np.median(filtered) + np.median(filtered) / 10000
elif thresh_type == "mean":
    threshold = np.mean(filtered) + np.std(filtered) * standards
elif thresh_type == "percentile":
    threshold = np.percentile(filtered, percentiles_seuil_r2)

# ========================
# BUILD results_dict
# ========================
results_dict = {
    'df': pd.DataFrame([{
        'subject': subject,
        'ch_names': channels,
        'scores': r2,
        'ch_names_masked': [
            "_" if score < threshold else ch 
            for score, ch in zip(r2, channels)
        ],
        'scores_masked': np.where(r2 < threshold, -np.inf, r2),
        'threshold': threshold
    }])
}

# ========================
# BUILD all_locs (3, n_channels)
# ========================
all_locs = locations_subject  # si un seul sujet
# ou bien np.concatenate([locations[subject]], axis=1) si tu veux garder la forme générale

# ========================
# EXPORT FINAL
# ========================
out_path = os.path.join(output_path, f'results_trf_{origin}_{montage_type}.pkl')
print("Exporting to:", out_path)

with open(out_path, 'wb') as f:
    pickle.dump([results_dict, all_locs], f)

print("✅ Export complet .pkl (results_dict + all_locs)")


NameError: name 'locations' is not defined

### Generate a .txt file with scores 

In [51]:
# Create the full path to the file
filename = os.path.join(out_path, 'scores.txt')

with open(filename, 'w') as file:
    # Write the header with aligned columns
    file.write(f'{"Channel":<10}{"Speech Acoustic (in %)":<25}{"Speech Predictif (in %)":<30}{"Music Acoustic (in %)":<30}{"Music Predictif (in %)":<30}\n')

    # Write data to the file
    for chan, scr1, scr2, scr3, scr4 in zip(sorted_channels_bipol, sorciere_regs_acoustic_scores_cv, sorciere_regs_predictive_scores_cv, pianos_regs_acoustic_scores_cv, pianos_regs_predictive_scores_cv):
        file.write(f'{chan:<15}{round(scr1*100,2):<30}{round(scr2*100,2):<25}{round(scr3*100,2):<30}{round(scr4*100,2):<20}\n')


## Plot results

### R2, PCI & ERP - with matlab

In [52]:
import matlab.engine
from datetime import datetime

print('Start at :', datetime.now())


# Démarrer le moteur MATLAB
eng = matlab.engine.start_matlab()
# run MATLAB script
eng.eval(f"out_path = '{out_path}'; run('C:/Users/nadege/Desktop/camille/seeg/agnes/sEEG_rsqr_pci.m')", nargout=0)
#eng.eval(f"out_path = '{out_path}'; run('//dynaserv/home/trebuchon/Desktop/Python_code/sEEG_rsqr_pci.m')", nargout=0)


print("End at:", datetime.now())

Start at : 2025-04-11 16:14:02.166881
End at: 2025-04-11 16:15:00.756938
